> Created by Teodora Szasz, tszasz@uchicago.edu

# <h1 align="center">2. Visualize reported incidents of crime in Chicago (4) </h1>

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# import Bokeh libraries
from bokeh.io import output_file, show
from bokeh.plotting import figure, ColumnDataSource

# import ColumnDataSource from Bokeh
from bokeh.models import Circle, Cross, Diamond

# categorical color mapper
from bokeh.transform import factor_cmap

# import Hover Tool
from bokeh.models import HoverTool

In [3]:
# give the capability of displaying multiple charts on the same page
from bokeh.layouts import column, row

In [4]:
# create output file
output_file('crimes_categiries_correlated.html')

In [5]:
# locate the file
file = '../Dataset/Crimes_-_2016_to_present.csv'

In [6]:
# use pandas' read_csv() method
crimes = pd.read_csv(file)

In [7]:
# option to be able to see all the columns
pd.set_option('display.max_columns', None)

In [8]:
crimes.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10842630,JA122203,01/19/2017 06:28:15 PM,059XX S ROCKWELL ST,2022,NARCOTICS,POSS: COCAINE,APARTMENT,True,False,824,8,16.0,66,18,NaN,NaN,2017,08/05/2017 03:50:08 PM,NaN,NaN,NaN
1,10842633,JA122240,01/19/2017 03:18:38 PM,006XX N ST LOUIS AVE,2024,NARCOTICS,POSS: HEROIN(WHITE),APARTMENT,True,False,1121,11,27.0,23,18,NaN,NaN,2017,08/05/2017 03:50:08 PM,NaN,NaN,NaN
2,10842652,JA122480,01/20/2017 12:52:00 AM,025XX E 106TH ST,2024,NARCOTICS,POSS: HEROIN(WHITE),RESIDENCE,True,False,434,4,10.0,51,18,NaN,NaN,2017,08/05/2017 03:50:08 PM,NaN,NaN,NaN
3,10885990,JA185288,03/13/2017 08:00:00 AM,033XX W FILLMORE ST,1812,NARCOTICS,POSS: CANNABIS MORE THAN 30GMS,POLICE FACILITY/VEH PARKING LOT,True,False,1134,11,24.0,29,18,NaN,NaN,2017,08/05/2017 03:50:08 PM,NaN,NaN,NaN
4,10886000,JA185322,03/13/2017 08:00:00 AM,033XX W FILLMORE ST,2024,NARCOTICS,POSS: HEROIN(WHITE),POLICE FACILITY/VEH PARKING LOT,True,False,1134,11,24.0,29,18,NaN,NaN,2017,08/05/2017 03:50:08 PM,NaN,NaN,NaN


In [9]:
# extract the District and Arrest columns
crimes_type_all = crimes[['District', 'Primary Type']]

In [10]:
# extract only the robberies
homicides = crimes_type_all.loc[crimes_type_all['Primary Type'] == 'HOMICIDE']

# extract only the robberies
robberies = crimes_type_all.loc[crimes_type_all['Primary Type'] == 'ROBBERY']

# extract only the assaults
assaults = crimes_type_all.loc[crimes_type_all['Primary Type'] == 'ASSAULT']

# extract only the thefts
narcotics = crimes_type_all.loc[crimes_type_all['Primary Type'] == 'NARCOTICS']

#count the number of homicides per district and remove the duplicates
homicides['Number_homicides'] = homicides.groupby('District')['District'].transform('count')
robberies['Number_robberies'] = robberies.groupby('District')['District'].transform('count')
narcotics['Number_narcotics'] = narcotics.groupby('District')['District'].transform('count')
assaults['Number_assaults'] = assaults.groupby('District')['District'].transform('count')
homicides = homicides.drop_duplicates('District')
robberies = robberies.drop_duplicates('District')
narcotics = narcotics.drop_duplicates('District')
assaults = assaults.drop_duplicates('District')

/Users/rcc/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rcc/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rcc/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [11]:
crimes_r_a_n = pd.merge(pd.merge(pd.merge(robberies, narcotics, on='District'), assaults, on = 'District'),
                        homicides, on='District')

In [12]:
crimes_r_a_n.head(5)

,District,Primary Type_x,Number_robberies,Primary Type_y,Number_narcotics,Primary Type_x,Number_assaults,Primary Type_y,Number_homicides
0,20,ROBBERY,132,NARCOTICS,91,ASSAULT,260,HOMICIDE,2
1,19,ROBBERY,460,NARCOTICS,122,ASSAULT,549,HOMICIDE,6
2,8,ROBBERY,845,NARCOTICS,419,ASSAULT,1148,HOMICIDE,45
3,7,ROBBERY,664,NARCOTICS,787,ASSAULT,1267,HOMICIDE,59
4,4,ROBBERY,597,NARCOTICS,417,ASSAULT,1348,HOMICIDE,52


In [13]:
districts=list(set(crimes_r_a_n['District']))
districts_str=[str(i) for i in districts]

In [14]:
crimes_r_a_n_data = ColumnDataSource(
    data=dict(
        robberies = list(crimes_r_a_n['Number_robberies']),
        narcotics = list(crimes_r_a_n['Number_narcotics']),
        assaults = list(crimes_r_a_n['Number_assaults']),
        homicides = list(crimes_r_a_n['Number_homicides']),
        districts=districts_str))

In [15]:
#generate some random colors for each district
N = len(districts)
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
z = np.random.random(size=N) * 100
colors = [
    "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b in zip(50+2*x, 30+2*y, 10+2*z)]

* Look at the number of homicides in each district

In [16]:
TOOLTIPS = 'pan, wheel_zoom, box_zoom, reset, save'

In [17]:
# create a figure object with 'tools' parameter defined
plot_r_n = figure(x_axis_label='Number of Robberies', y_axis_label='Number of Narcotics', 
              tools = TOOLTIPS,
              title = 'Number of Robberies versus Number of Narcotics in Chicago')

In [18]:
plot_r_a = figure(x_axis_label='Number of Robberies', y_axis_label='Number of Assaults', 
              tools = TOOLTIPS,
              title = 'Number of Robberies versus Number of Assaults in Chicago')

In [19]:
plot_r_h = figure(x_axis_label='Number of Robberies', y_axis_label='Number of Homicides', 
              tools = TOOLTIPS,
              title = 'Number of Robberies versus Number of Homicides in Chicago')

http://bokeh.pydata.org/en/latest/docs/reference/models/markers.html

In [20]:
# use circle glyph for our robberies_versus_narcotics figure object

circle=Circle(x="robberies", y="narcotics", size=20, 
              fill_color=factor_cmap('districts', palette=colors, factors=districts_str), fill_alpha=0.5)
plot_r_n.add_glyph(crimes_r_a_n_data,circle)

GlyphRenderer(id='bb4a5c50-9959-4e2a-8f1d-65a9e0178b95', ...)

In [21]:
# use circle glyph for our narcotics_versus_assaults figure object

diamond=Diamond(x="narcotics", y="assaults", size=20, 
              fill_color=factor_cmap('districts', palette=colors, factors=districts_str), fill_alpha=0.5)
plot_r_a.add_glyph(crimes_r_a_n_data,diamond)

GlyphRenderer(id='335e9486-fdc2-49c0-aa59-545d4e432180', ...)

In [22]:
# use circle glyph for our robberies_versus_assaults figure object

cross=Cross(x="robberies", y="homicides", size=20, 
              fill_color=factor_cmap('districts', palette=colors, factors=districts_str), fill_alpha=0.5)
plot_r_h.add_glyph(crimes_r_a_n_data,cross)

GlyphRenderer(id='2811d9a0-f414-4ac5-bb18-1653e6255221', ...)

In [23]:
# customize the legend
plot_r_n.legend.location = 'bottom_right'
plot_r_n.legend.background_fill_color = 'lightgrey'

In [24]:
# customize the legend
plot_r_a.legend.location = 'bottom_right'
plot_r_a.legend.background_fill_color = 'lightgrey'

In [25]:
# customize the legend
plot_r_h.legend.location = 'bottom_right'
plot_r_h.legend.background_fill_color = 'lightgrey'

In [26]:
# customize the Hover tool
hover_r_n = HoverTool(tooltips = 
                  [("District Number", "@districts"),
                 ("Number of Robberies", "@robberies"),
                ("Number of Narcotics", "@narcotics"),])
plot_r_n.add_tools(hover_r_n)

In [27]:
# customize the Hover tool
hover_r_a = HoverTool(tooltips = 
                  [("District Number", "@districts"),
                 ("Number of Robberies", "@robberies"),
                ("Number of Assaults", "@assaults"),])
plot_r_a.add_tools(hover_r_a)

In [28]:
# customize the Hover tool
hover_r_h = HoverTool(tooltips = 
                  [("District Number", "@districts"),
                 ("Number of Robberies", "@robberies"),
                ("Number of Homicides", "@homicides"),])
plot_r_h.add_tools(hover_r_h)

In [29]:
# show the result
#show(row(plot_r_n, plot_r_a, plot_r_h))

In [30]:
# correlate the plots with each other - 3rd phase
plot_r_a.x_range = plot_r_n.x_range
plot_r_h.x_range = plot_r_n.x_range

In [31]:
# show the result - 2nd phase
show(row(column(plot_r_n, plot_r_a),column(plot_r_h)))

In [34]:
from bokeh.io import output_notebook
output_notebook()
show(row(column(plot_r_n, plot_r_a),column(plot_r_h)))

Loading BokehJS ...